In [1]:
import torch
import os
import cv2
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
from torchvision import transforms
from tqdm import tqdm
PROJECT_NAME = "Intel-Classification-V3"
device = 'cuda'

In [2]:
transformation_data = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

In [3]:
img = cv2.imread('./data/buildings/0.jpg')

In [4]:
img_pil_image = Image.fromarray(img)
# transformation_data(img_pil_image)

In [5]:
np.eye(5,5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [6]:
def load_image(data_dir="./data/"):
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for label in os.listdir(data_dir):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label
    for directory in tqdm(os.listdir(data_dir)):
        for file_dir in os.listdir(data_dir + directory):
            img = cv2.imread(data_dir + directory + "/" + file_dir)
            img = cv2.resize(img,(56,56))
            data.append([np.array(transformation_data(Image.fromarray(img))),np.eye(labels[directory]+1,idx+1)[-1]])
    np.random.shuffle(data)
    data = data[:5000]
    X = []
    y = []
    for d_iter in data:
        X.append(d_iter[0])
        y.append(d_iter[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=True)
    X_train = torch.from_numpy(np.array(X_train)).to(device)
    y_train = torch.from_numpy(np.array(y_train)).to(device)
    X_test = torch.from_numpy(np.array(X_test)).to(device)
    y_test = torch.from_numpy(np.array(y_test)).to(device)
    return labels,labels_r,data,X,y,X_train,X_test,y_train,y_test,idx

In [7]:
labels,labels_r,data,X,y,X_train,X_test,y_train,y_test,idx = load_image()

In [8]:
torch.save(labels,'./save/labels.pt')
torch.save(labels,'./save/labels.pth')
torch.save(labels_r,'./save/labels_r.pt')
torch.save(labels_r,'./save/labels_r.pth')
torch.save(data,'./save/data.pt')
torch.save(data,'./save/data.pth')
torch.save(X,'./save/X.pt')
torch.save(y,'./save/y.pth')
torch.save(X_train,'./save/X_train.pt')
torch.save(X_test,'./save/X_test.pth')
torch.save(y_train,'./save/y_train.pt')
torch.save(y_test,'./save/y_test.pth')

In [9]:
len(X_train),len(X_test),len(y_train),len(y_test)

(3750, 1250, 3750, 1250)

In [10]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [11]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for y_iter,pred in zip(y,preds):
        y_iter = torch.argmax(y_iter)
        pred = torch.argmax(pred)
        if pred == y_iter:
            correct += 1
        total += 1
    return round(correct/total,3)

In [12]:
from torch.nn import *

In [13]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,9,3)
        self.conv2 = Conv2d(9,12,3)
        self.convbatch_norm3 = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,3)
        self.conv5 = Conv2d(15,18,3)
        self.linear1 = Linear(18*12*12,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)))
        preds = self.max_pool2d(self.relu(self.conv2(preds)))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)))
        preds = self.max_pool2d(self.relu(self.conv5(preds)))
        print(preds.shape)
        preds = preds.view(-1,18*12*12)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [14]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [15]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [16]:
import wandb

In [17]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [18]:
!pip3 install wandb

In [19]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [20]:
import wandb

In [21]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [22]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,9,3)
        self.conv2 = Conv2d(9,12,3)
        self.convbatch_norm3 = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,3)
        self.conv5 = Conv2d(15,18,3)
        self.linear1 = Linear(18*12*12,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(3,3))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(3,3))
        print(preds.shape)
        preds = preds.view(-1,18*12*12)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [23]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [24]:
import wandb

In [25]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [26]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,9,3)
        self.conv2 = Conv2d(9,12,3)
        self.convbatch_norm3 = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,3)
        self.conv5 = Conv2d(15,18,3)
        self.linear1 = Linear(18*12*12,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(3,3))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(1,1))
        print(preds.shape)
        preds = preds.view(-1,18*12*12)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [27]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [28]:
import wandb

In [29]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [30]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,9,3)
        self.conv2 = Conv2d(9,12,3)
        self.convbatch_norm3 = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,3)
        self.conv5 = Conv2d(15,18,3)
        self.linear1 = Linear(18*12*12,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(3,3))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(3,3))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,18*12*12)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [31]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [32]:
import wandb

In [33]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [34]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,8,3)
        self.conv2 = Conv2d(8,16,3)
        self.convbatch_norm3 = BatchNorm2d(16)
        self.conv4 = Conv2d(16,24,3)
        self.conv5 = Conv2d(24,32,3)
        self.linear1 = Linear(18*12*12,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(2,2))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,18*12*12)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [35]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [36]:
import wandb

In [37]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [38]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,8,3)
        self.conv2 = Conv2d(8,16,3)
        self.convbatch_norm3 = BatchNorm2d(16)
        self.conv4 = Conv2d(16,24,3)
        self.conv5 = Conv2d(24,32,3)
        self.linear1 = Linear(32*1*1,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(2,2))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,32*1*1)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [39]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [40]:
import wandb

In [41]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [42]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,8,5)
        self.conv2 = Conv2d(8,16,5)
        self.convbatch_norm3 = BatchNorm2d(16)
        self.conv4 = Conv2d(16,24,5)
        self.conv5 = Conv2d(24,32,5)
        self.linear1 = Linear(32*1*1,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(2,2))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,32*1*1)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [43]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [44]:
import wandb

In [45]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [46]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,8,2)
        self.conv2 = Conv2d(8,16,2)
        self.convbatch_norm3 = BatchNorm2d(16)
        self.conv4 = Conv2d(16,24,2)
        self.conv5 = Conv2d(24,32,2)
        self.linear1 = Linear(32*1*1,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(2,2))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,32*1*1)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [47]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [48]:
import wandb

In [49]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [50]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = F.max_pool2d
        self.relu = ReLU()
        self.conv1 = Conv2d(3,8,2)
        self.conv2 = Conv2d(8,16,2)
        self.convbatch_norm3 = BatchNorm2d(16)
        self.conv4 = Conv2d(16,24,2)
        self.conv5 = Conv2d(24,32,2)
        self.linear1 = Linear(32*2*2,128)
        self.linear2 = Linear(128,256)
        self.linearbatchnorm3 = BatchNorm1d(256)
        self.linear4 = Linear(256,128)
        self.linear5 = Linear(128,idx)
    
    def forward(self,X):
        preds = self.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv2(preds)),(2,2))
        preds = self.convbatch_norm3(preds)
        preds = self.max_pool2d(self.relu(self.conv4(preds)),(2,2))
        preds = self.max_pool2d(self.relu(self.conv5(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,32*2*2)
        preds = self.relu(self.linear1(preds))
        preds = self.relu(self.linear2(preds))
        preds = self.linearbatchnorm3(preds)
        preds = self.relu(self.linear4(preds))
        preds = self.relu(self.linear5(preds))
        return preds

In [51]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [52]:
import wandb

In [53]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [54]:
preds

tensor([], device='cuda:0', size=(32, 0), grad_fn=<ReluBackward0>)

In [55]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*10*10,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,50)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        preds = preds.view(-1,128*10*10)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [56]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [57]:
import wandb

In [58]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [59]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*10*10,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,50)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,128*10*10)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [60]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [61]:
import wandb

In [62]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [63]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,50)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,128*3*3)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [64]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [65]:
import wandb

In [66]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [67]:
preds

tensor([[0.1338, 0.0000, 0.0126,  ..., 0.0000, 0.0000, 0.0000],
        [0.1402, 0.0000, 0.0558,  ..., 0.0000, 0.0000, 0.0167],
        [0.1271, 0.0000, 0.0295,  ..., 0.0000, 0.0000, 0.0224],
        ...,
        [0.1804, 0.0000, 0.0185,  ..., 0.0000, 0.0000, 0.0000],
        [0.1328, 0.0179, 0.0070,  ..., 0.0000, 0.0000, 0.0000],
        [0.1364, 0.0033, 0.0208,  ..., 0.0000, 0.0000, 0.0199]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [68]:
y_batch

tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0

In [69]:
preds[0]

tensor([0.1338, 0.0000, 0.0126, 0.0000, 0.0285, 0.0414, 0.0000, 0.0625, 0.0923,
        0.0627, 0.0000, 0.0000, 0.0800, 0.0000, 0.1386, 0.0808, 0.0000, 0.0000,
        0.0000, 0.1432, 0.0827, 0.0053, 0.0000, 0.0000, 0.0149, 0.0315, 0.0482,
        0.0000, 0.0358, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1098,
        0.0120, 0.0023, 0.0683, 0.0648, 0.0624, 0.0517, 0.0000, 0.1061, 0.0667,
        0.0389, 0.0219, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [70]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,6)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        print(preds.shape)
        preds = preds.view(-1,128*3*3)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [71]:
model = Model().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 100

In [72]:
import wandb

In [73]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })

In [74]:
wandb.init(project=PROJECT_NAME,name="BaseLine")
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,56,56).to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds.float(),y_batch.float())
        optimizer.zero_grad()
        optimizer.step()
        loss.backward()
        wandb.log({
            "Test Loss":get_loss(model,X_test,y_test),
            "Test Accuracy":get_accuracy(model,X_test,y_test),
            "Accuracy":get_accuracy(model,X_batch,y_batch),
            "Loss":get_loss(model,X_batch,y_batch)
        })